In [1]:
import pandas as pd 
from config import username, password, username_heroku, password_heroku, database_heroku, host_heroku, gkey
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv("Assets/Data/usa_power_plant_database.csv")
df

,id,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,...,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,estimated_generation_gwh
0,1,USA,United States of America,12 Applegate Solar LLC,USA0059371,1.9,40.2003,-74.5761,Solar,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,NaN,2017.0,2.41461,2.35000,2.430000,2.492000,2.27600,NaN
1,2,USA,United States of America,126 Grove Solar LLC,USA0060858,2.0,42.0761,-71.4227,Solar,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,NaN,2017.0,NaN,NaN,NaN,2.416000,2.25100,0.891855
2,3,USA,United States of America,1420 Coil Av #C,USA0057310,1.3,33.7943,-118.2414,Solar,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,NaN,2017.0,1.52800,2.14900,1.515000,1.592000,1.66000,NaN
3,4,USA,United States of America,145 Talmadge Solar,USA0057458,3.8,40.5358,-74.3913,Solar,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,67644.0,2017.0,5.03600,4.52400,4.802000,5.051000,4.81900,NaN
4,5,USA,United States of America,1515 S Caron Road,USA0007770,4.2,41.9084,-89.0466,Gas,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,52308.0,2017.0,0.21500,0.17800,0.271000,0.306000,0.26400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,8686,USA,United States of America,Zion Energy Center,USA0055392,596.7,42.4776,-87.8950,Gas,Oil,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,49140.0,2017.0,0.00000,63.65600,132.434005,435.493999,462.06300,NaN
8686,8687,USA,United States of America,Zion Landfill Gas to Energy Facility,USA0056871,7.0,42.4803,-87.8861,Waste,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,50113.0,2017.0,43.59200,49.13400,46.699000,42.480000,38.66900,NaN
8687,8688,USA,United States of America,Zorn,USA0001368,18.0,38.2803,-85.7023,Gas,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,25154.0,2017.0,0.20300,0.07800,1.058000,0.062000,0.01500,NaN
8688,8689,USA,United States of America,Zotos International WPGF,USA0057648,3.4,42.8869,-76.9683,Wind,NaN,...,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,NaN,2017.0,3.67100,2.46015,2.489000,1.670000,1.23349,NaN


In [3]:
del df["other_fuel1"]
del df["other_fuel2"]
del df["other_fuel3"]

In [4]:
del df["estimated_generation_gwh"]
del df["generation_gwh_2013"]
del df["generation_gwh_2014"]
del df["generation_gwh_2015"]
del df["generation_gwh_2016"]

In [5]:
for column in df:
    nullVal = df[column].isnull().sum()
    print(f"{column}: {nullVal} // {round(nullVal / len(df) *100,2)}")

id: 0 // 0.0
country: 0 // 0.0
country_long: 0 // 0.0
name: 0 // 0.0
gppd_idnr: 0 // 0.0
capacity_mw: 0 // 0.0
latitude: 0 // 0.0
longitude: 0 // 0.0
primary_fuel: 0 // 0.0
commissioning_year: 21 // 0.24
owner: 13 // 0.15
source: 0 // 0.0
url: 0 // 0.0
geolocation_source: 0 // 0.0
wepp_id: 3468 // 39.91
year_of_capacity_data: 42 // 0.48
generation_gwh_2017: 151 // 1.74


In [6]:
fuel_types = df["primary_fuel"].unique()
fuel_types

array(['Solar', 'Gas', 'Oil', 'Hydro', 'Wind', 'Coal', 'Biomass', 'Waste',
       'Cogeneration', 'Storage', 'Geothermal', 'Nuclear', 'Petcoke',
       'Other'], dtype=object)

In [7]:
for fuel in fuel_types:
    new_df = df.loc[df['primary_fuel'] == fuel]
    avg = new_df["generation_gwh_2017"].mean()
    print(f"{fuel}: {round(avg,3)}")
    df['generation_gwh_2017'] = df['generation_gwh_2017'].fillna(avg)

Solar: 23.422
Gas: 749.45
Oil: 23.506
Hydro: 201.838
Wind: 243.609
Coal: 3548.375
Biomass: 174.993
Waste: 83.634
Cogeneration: 114.993
Storage: 0.909
Geothermal: 244.7
Nuclear: 13359.418
Petcoke: 694.029
Other: 102.487


In [8]:
df["owner"] = df["owner"].fillna("Not listed")
df["commissioning_year"] = df["commissioning_year"].fillna(0)
df["year_of_capacity_data"] = 2017
df["wepp_id"] = df["wepp_id"].fillna(0)

In [9]:
for column in df:
    nullVal = df[column].isnull().sum()
    print(f"{column}: {nullVal} // {round(nullVal / len(df) *100,2)}%")

id: 0 // 0.0%
country: 0 // 0.0%
country_long: 0 // 0.0%
name: 0 // 0.0%
gppd_idnr: 0 // 0.0%
capacity_mw: 0 // 0.0%
latitude: 0 // 0.0%
longitude: 0 // 0.0%
primary_fuel: 0 // 0.0%
commissioning_year: 0 // 0.0%
owner: 0 // 0.0%
source: 0 // 0.0%
url: 0 // 0.0%
geolocation_source: 0 // 0.0%
wepp_id: 0 // 0.0%
year_of_capacity_data: 0 // 0.0%
generation_gwh_2017: 0 // 0.0%


In [10]:
df['commissioning_year'] = df['commissioning_year'].astype(int)

In [11]:
df.set_index("id",inplace=True)

In [13]:
df["state"] = ""

In [14]:
df

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2017,state
id,,,,,,,,,,,,,,,,,
1,USA,United States of America,12 Applegate Solar LLC,USA0059371,1.9,40.2003,-74.5761,Solar,2012,SunRay Power LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,2.27600,
2,USA,United States of America,126 Grove Solar LLC,USA0060858,2.0,42.0761,-71.4227,Solar,2012,126 Grove Solar LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,2.25100,
3,USA,United States of America,1420 Coil Av #C,USA0057310,1.3,33.7943,-118.2414,Solar,2011,Konoike Pacific,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,1.66000,
4,USA,United States of America,145 Talmadge Solar,USA0057458,3.8,40.5358,-74.3913,Solar,2011,Avidan Energy Solutions,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,67644.0,2017,4.81900,
5,USA,United States of America,1515 S Caron Road,USA0007770,4.2,41.9084,-89.0466,Gas,2000,Rochelle Municipal Utilities,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,52308.0,2017,0.26400,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8686,USA,United States of America,Zion Energy Center,USA0055392,596.7,42.4776,-87.8950,Gas,2002,Zion Energy LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,49140.0,2017,462.06300,
8687,USA,United States of America,Zion Landfill Gas to Energy Facility,USA0056871,7.0,42.4803,-87.8861,Waste,2003,Energy Developments Inc,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,50113.0,2017,38.66900,
8688,USA,United States of America,Zorn,USA0001368,18.0,38.2803,-85.7023,Gas,1969,Louisville Gas & Electric Co,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,25154.0,2017,0.01500,


In [ ]:
# # Create connection (local)
# connection_string = f"{username}:{password}@localhost:5432/usa_power_plants"
# engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# # Confirm tables
# engine.table_names()

In [ ]:
# # Convert df into sql db
# df.to_sql(name='power_plants', con=engine, if_exists='replace',index=False)

In [ ]:
# # Create connection (heroku)
# connection_string = f"{username_heroku}:{password_heroku}@{host_heroku}:5432/{database_heroku}"
# engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# # Confirm tables
# engine.table_names()

In [ ]:
# # Convert df into sql db
# df.to_sql(name='power_plants', con=engine, if_exists='replace')

In [ ]:
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy import Column, Integer, String, Float

In [ ]:
# Base = declarative_base()

In [ ]:
# class Power_plants(Base):
#     __tablename__ = "power_plants"
#     id = Column(Integer, primary_key=True)
#     country = Column(String)
#     country_long = Column(String)
#     name = Column(String)
#     gppd_idnr = Column(String)
#     capacity_mw = Column(Float)
#     latitude = Column(Float)
#     longitude = Column(Float)
#     primary_fuel = Column(String)
#     commissioning_year = Column(Integer)
#     owner = Column(String)
#     source = Column(String)
#     url = Column(String)
#     geolocation_source = Column(String)
#     wepp_id = Column(Integer)
#     year_of_capacity_data = Column(Float)
#     generation_gwh_2017 = Column(Float)

In [ ]:
# session = Session(bind=engine)

In [ ]:
# session.execute("select * from power_plants")

In [ ]:
# for i, row in df.iterrows():
#     power_plant = Power_plants(id=row["id"], country=row["country"], country_long=row["country_long"], name=row["name"], 
#                               gppd_idnr=row["gppd_idnr"], capacity_mw=row["capacity_mw"], latitude=row["latitude"], longitude=row["longitude"],
#                               primary_fuel=row["primary_fuel"], commissioning_year=row["commissioning_year"], owner=row["owner"], source=row["source"],
#                               url=row["url"], geolocation_source=row["geolocation_source"], wepp_id=row["wepp_id"], year_of_capacity_data=row["year_of_capacity_data"],
#                               generation_gwh_2017=row["generation_gwh_2017"])
#     session.add(power_plant)

In [15]:
import requests
import json

In [17]:
# url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={gkey}"

# lat = 39.5952
# lng = -78.7453

# geo_data = requests.get(url).json()

# print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "plus_code": {
        "compound_code": "H7W3+3V North Branch, MD, USA",
        "global_code": "87F3H7W3+3V"
    },
    "results": [
        {
            "address_components": [
                {
                    "long_name": "11600",
                    "short_name": "11600",
                    "types": [
                        "street_number"
                    ]
                },
                {
                    "long_name": "Mexico Farm Road Southeast",
                    "short_name": "Mexico Farm Rd SE",
                    "types": [
                        "route"
                    ]
                },
                {
                    "long_name": "Cumberland",
                    "short_name": "Cumberland",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "16, North Branch",
                    "short_name

In [19]:
# geo_data["results"][0]['address_components'][4]['long_name']

'Maryland'

In [24]:
# for index, row in df.iloc[:10].iterrows():
for index, row in df.iterrows():
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={gkey}"

        lat = row['latitude']
        lng = row['longitude']

        geo_data = requests.get(url).json()
        
        response = geo_data["results"][0]['address_components'][4]['long_name']
        
        if "County" in response:
            df.loc[index, "state"] = geo_data["results"][0]['address_components'][5]['long_name']
        else:
            df.loc[index, "state"] = geo_data["results"][0]['address_components'][4]['long_name']
    except:
        print(f"{row['name']} --> not found")

AES Warrior Run Cogeneration Facility --> not found
Ainsworth Wind --> not found
Alder --> not found
American Crystal Sugar Crookston --> not found
Amoskeag --> not found
Anchorage 1 --> not found
Ansley --> not found
Bearford Farm Solar Project --> not found
Berlin --> not found
Bloomfield --> not found
Blue Canyon Windpower II --> not found
Blue Canyon Windpower V LLC --> not found
Blythe Energy Inc --> not found
Boardman --> not found
Bonny Eagle --> not found
Boulder --> not found
Box Canyon --> not found
Brandywine Power Facility --> not found
Buck Hydro --> not found
Buckingham Solar LLC --> not found
CC Perry K --> not found
Carmi --> not found
Cedar Bayou 4 --> not found
Cedar Creek --> not found
Chuckawalla Solar --> not found
Clark (SD) --> not found
Colstrip --> not found
Coso Finance Partners --> not found
Costa Sur --> not found
Crescent Ridge --> not found
Dane County Landfill #2 Rodefeld --> not found
Deep Branch Farm --> not found
Don A Campbell 2 Geothermal --> not fou

In [26]:
df.head(50)

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,commissioning_year,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_gwh_2017,state
id,,,,,,,,,,,,,,,,,
1,USA,United States of America,12 Applegate Solar LLC,USA0059371,1.9,40.2003,-74.5761,Solar,2012,SunRay Power LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,2.276000,Massachusetts
2,USA,United States of America,126 Grove Solar LLC,USA0060858,2.0,42.0761,-71.4227,Solar,2012,126 Grove Solar LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,2.251000,New Jersey
3,USA,United States of America,1420 Coil Av #C,USA0057310,1.3,33.7943,-118.2414,Solar,2011,Konoike Pacific,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,1.660000,Massachusetts
4,USA,United States of America,145 Talmadge Solar,USA0057458,3.8,40.5358,-74.3913,Solar,2011,Avidan Energy Solutions,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,67644.0,2017,4.819000,California
5,USA,United States of America,1515 S Caron Road,USA0007770,4.2,41.9084,-89.0466,Gas,2000,Rochelle Municipal Utilities,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,52308.0,2017,0.264000,New Jersey
6,USA,United States of America,158th Fighter Wing Solar Farm,USA0060542,1.3,44.4777,-73.1534,Solar,2011,158th Fighter Wing,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,1.616000,Illinois
7,USA,United States of America,180 Raritan Solar,USA0058187,1.9,40.5161,-74.3400,Solar,2012,180 Raritan Energy Solutions LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,67643.0,2017,2.547000,Vermont
8,USA,United States of America,191 Peachtree Tower,USA0054818,2.4,33.7583,-84.3869,Oil,1991,Banyan St./GAP 191 Peachtree Owner LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,48724.0,2017,0.000000,New Jersey
9,USA,United States of America,201 Sturbridge B,USA0059822,2.0,42.1091,-72.1712,Solar,2014,SE Solarne2 LLC,U.S. Energy Information Administration,http://www.eia.gov/electricity/data/browser/,U.S. Energy Information Administration,0.0,2017,2.722000,Georgia


In [ ]:
# We could not get the df to sql  to work with the heroku db so we used the csv file we just created 
# from the data manipulation and loaded that directly into our postgres db
df.to_csv("Assets/Data/clean_USA_power_plant_data.csv")